In [ ]:
import numpy as np
from scipy.spatial.distance import cdist
import pandas as pd
import copy
from itertools import cycle, islice
from sklearn import cluster, datasets, mixture
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
plt.style.use('default')

# Begin here to import PCA and UMAP and Leiden Cluster Labels

In [ ]:
pca_analysis = pd.read_csv('pca_results.csv', header = 0)
pca_analysis

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
pcx = '0'
pcy = '1'
ax.scatter(pca_analysis[pcx], pca_analysis[pcy])
ax.set_xlabel(pcx)
ax.set_ylabel(pcy)
plt.show()

In [ ]:
n_pcs = 7
values_matrix = pca_analysis.values[:, 0:n_pcs]
print(values_matrix.shape)

In [ ]:
import numpy as np

# Assuming values_matrix is your data matrix

mean = np.mean(values_matrix, axis=0)
std_dev = np.std(values_matrix, axis=0)

# Calculate the bounds for 3 standard deviations
lower_bound = mean - 3 * std_dev
upper_bound = mean + 3 * std_dev

# Create 23 bins within the bounds and add 2 bins for the outliers
bins = np.empty((10, values_matrix.shape[1]))
for col in range(values_matrix.shape[1]):
    bins[1:-1, col] = np.linspace(lower_bound[col], upper_bound[col], num=8)
    bins[0, col] = -np.inf  # Bin for values below lower bound
    bins[-1, col] = np.inf  # Bin for values above upper bound

# Digitize the values
digitized = np.empty_like(values_matrix)
for col in range(values_matrix.shape[1]):
    digitized[:, col] = np.digitize(values_matrix[:, col], bins=bins[:, col])




In [ ]:
# digitized now contains the indices of the bins to which each value belongs
print(np.shape(bins))
print(np.shape(digitized))

In [ ]:
#count the number of points in each bin combo. We have 25 bins for each x,y
dict25 = {}
for row in digitized:
    str_row = ' '.join(map(str, row.astype(int)))
    if  str_row not in dict25.keys():
        dict25[str_row] = 1
    else:
        dict25[str_row] = dict25[str_row] + 1

In [ ]:
len(dict25)

In [ ]:
#convert to probabilities
total_counts = sum(dict25.values())
dict25_sp = {}
for k, v in dict25.items():
    dict25_sp[k] = v / total_counts

In [ ]:
# We have to sort it from highest probability to lowest in the txt output

# Sort the dictionary by value in descending order
sorted_dict = dict(sorted(dict25_sp.items(), key=lambda x: x[1], reverse=True))
#sorted_dict

## Clustering Algorithm Results added here

In [ ]:
# this is the clustering algorithm read in from the txt file vector data is the dimension space after clustering
microstates = pd.read_csv( "scanpy_pcs.txt.negmap" , sep="|" , skiprows= [1])
microstates.columns = [col.strip() for col in microstates.columns]
microstates["Vector"] = microstates["Vector"].apply(lambda v: np.array(v.strip().strip("[|]").split(), dtype= int))
microstates.head()

In [ ]:
microstates.shape

done to ensure original files are not modified

In [ ]:
#find unique Pk in microstates
unique_pk_micro = microstates["Pk"].unique()
print(len(unique_pk_micro))

In [ ]:
# Finding the highest and lowest probabilities
highest_prob = microstates['Prob'].max()
lowest_prob = microstates['Prob'].min()

highest_prob, lowest_prob


### Scanpy_pcs

this is the another file not sure what it is but it probably has the clusters birth and death info again 

In [ ]:
#Now we want to relabel the Pk value of the Pk centers because we had to kill them during persistent homology algorithm
#this is the proabalistic map, the 0.000379075 means 1/2638
peaks = pd.read_csv("scanpy_pcs.txt" , sep="|" , skiprows= [1])
peaks.columns = [col.strip() for col in peaks.columns]


#relabel the Pk values using Birth State Index
clusters_ids = peaks["Birth State Index"].unique()

for peak in clusters_ids:
    microstates['Pk'][peak] = peak


In [ ]:
#  find unique Pk again after relabeling
unique_pk = microstates["Pk"].unique()
print(len(unique_pk))

In [ ]:
# group dataframe by "Pk" and sum the "Prob" column for each group
cluster_probs = microstates.groupby('Pk')['Prob'].sum().to_dict()
cluster_probs = dict(sorted(cluster_probs.items(), key=lambda x: x[1])) #, reverse = True


In [ ]:
# Function to find Pk value for a given vector element
def find_pk_for_vector_element(dataframe, vector_element):
    for index, row in dataframe.iterrows():
        if np.array_equal(row['Vector'], vector_element):
            return row['Pk']
    return None

pks_for_data = []
for i in range(len(digitized)):
    pks_for_data.append(find_pk_for_vector_element(microstates, digitized[i].astype(int)))

In [ ]:
print(pks_for_data)

In [ ]:
#  find unique Pk again after relabeling
unique_pk = microstates["Pk"].unique()
print(len(unique_pk))

In [ ]:
from collections import Counter

frequency = Counter(pks_for_data)

# Extract keys where values are less than 5
keys_less_than = [key for key, value in frequency.items() if value < 5] #ari = 0.89 for <5

In [ ]:
#based on this keeping the pk with highest probability
#making a new df to store the aggregated data
aggregated_data = microstates.groupby('Pk').apply(lambda x: x.loc[x['Prob'].idxmax()]).reset_index(drop=True)

# Displaying the first few rows of the new aggregated DataFrame with max probability
aggregated_data.head()

In [ ]:
# find unique Pk again after relabeling
unique_pk_agg = aggregated_data["Pk"].unique()
print(len(unique_pk_agg))

In [ ]:
# aggregated_data shape
aggregated_data.shape

In [ ]:
# save the aggregated data to a csv file
aggregated_data.to_csv('aggregated_data.csv', index=False)

same shape as the unique vales

In [ ]:
# Merging 'microstates.csv' and 'peaks.csv' on 'Pk' and 'Birth State Index'
merged_data = pd.merge(aggregated_data, peaks, left_on='Pk', right_on='Birth State Index', how='left')

# Displaying the first few rows of the merged DataFrame
print(merged_data.shape)
merged_data.head()

In [ ]:
# find missing values in the merged data
merged_data.isnull().sum()

In [ ]:
# Are Prob and Birth Probability the same for each row?
merged_data['Prob'].equals(merged_data['Birth Probability'])


In [ ]:
# unique values Pk in merged data
unique_pk_merged = merged_data["Pk"].unique()
print(len(unique_pk_merged))

making new dataframe with pk values labelled for each row to the pca

In [ ]:
# Add the Pk values as a new column
pca_analysis['Pk'] = pks_for_data  # pks_for_data should be a list with the same length as pca_analysis

# Make 'Pk' the first column by reordering the columns
cols = ['Pk'] + [col for col in pca_analysis.columns if col != 'Pk']
pca_analysis = pca_analysis[cols]

# save the pca_analysis to a csv file
pca_analysis.to_csv('pca_analysis.csv', index=False)

In [ ]:
# Display the first few rows of the DataFrame to verify the column order
pca_analysis.head()


In [ ]:
# find the unique Pk in pca_analysis
unique_pk_pca = pca_analysis["Pk"].unique()
print(len(unique_pk_pca))


I did a couple pf merges and this is the only one that Birth Probability and the Prob in both columns match 

In [ ]:
# save the merged data to a csv file
merged_data.to_csv('merged_data.csv', index=False)

# load the merged data from the csv file
final_data = pd.read_csv('merged_data.csv')
final_data.head()

Since the 'Vector' column represents the dimensions made during clustering, we could consider each element of these vectors as features.
we can then compute the Pearson correlation matrix. This matrix will show the correlation between each pair of features across all clusters.

### Correlation Matrix

In [ ]:
import ast

# Check the number of unique Pk values to ensure there are 379
num_unique_pks = pca_analysis['Pk'].nunique()
print(num_unique_pks)

# For each Pk, compute the mean of the vector dimensions
pk_means = pca_analysis.groupby('Pk').mean()

# We need a correlation matrix where each Pk value correlates with every other Pk value
# To achieve this, we use the transpose of the pk_means dataframe to compute the correlation
# This will treat each Pk as a separate entity and compute correlations between their mean vector values
pk_correlation_matrix_final = pk_means.T.corr()


In [ ]:
import ast

# Convert the 'Vector' column into actual lists
final_data['Vector'] = final_data['Vector'].apply(lambda x: [int(num) for num in x.strip('[]').split()])

# Convert the 'Vector' lists into separate columns
vector_df = pd.DataFrame(final_data['Vector'].tolist())

# Incorporate the 'Prob' column if it's needed for the analysis
# For now, we'll focus on the vector data
# Combine the 'Pk' column with the vector dataframe
combined_df = pd.concat([final_data[['Pk', 'Prob']], vector_df], axis=1)

# To ensure Pk values are treated as unique identifiers (like strings), we'll convert them to string type
#combined_df['Pk'] = combined_df['Pk'].astype(str)

# Check the number of unique Pk values to ensure there are 309
num_unique_pks = combined_df['Pk'].nunique()
print(num_unique_pks)

# For each Pk, compute the mean of the vector dimensions
pk_means = combined_df.groupby('Pk').mean()

# We need a correlation matrix where each Pk value correlates with every other Pk value
# To achieve this, we use the transpose of the pk_means dataframe to compute the correlation
# This will treat each Pk as a separate entity and compute correlations between their mean vector values
pk_correlation_matrix_final = pk_means.T.corr()


In [ ]:
pk_correlation_matrix_final

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the size of the heatmap
plt.figure(figsize=(20, 15))

# Create a heatmap to visualize the correlation matrix
sns.heatmap(pk_correlation_matrix_final, cmap='coolwarm')

# Set the title of the heatmap
plt.title('Heatmap of Pearson Correlation between Pk Values')

# Show the heatmap
plt.show()


Let us try this for blocks that are less than 5/2638

In [ ]:
# Calculate the threshold probability value
threshold_prob = 5 / 2638

# Filter out the rows where the probability is less than the threshold
filtered_df = combined_df[combined_df['Prob'] < threshold_prob]

# For the filtered Pk values, create a new dataframe for correlation calculation
# We'll use the mean of the vector dimensions for each Pk, as we did before
filtered_pk_means = filtered_df.groupby('Pk').mean()

# Compute the Pearson correlation matrix for this filtered set of Pk values
filtered_pk_correlation_matrix = filtered_pk_means.T.corr()

# Display the shape of the new correlation matrix to confirm its size
print(filtered_pk_correlation_matrix.shape)

filtered_pk_correlation_matrix.head()  # Displaying part of the matrix for a glimpse


In [ ]:
# Create a heatmap for the filtered correlation matrix
plt.figure(figsize=(20, 15))
sns.heatmap(filtered_pk_correlation_matrix, cmap='coolwarm')
plt.title('Heatmap of Pearson Correlation for Filtered Pk Values (Prob < 5/2638)')
plt.show()


In [ ]:
# save the correlation matrix to a csv file
filtered_pk_correlation_matrix.to_csv('filtered_pk_correlation_matrix.csv')


In [ ]:
# Calculate the threshold probability value
threshold_prob = 5 / 2638

# Filter out the rows where the probability is less than the threshold
low_prob_pks = final_data[final_data['Prob'] < threshold_prob]

# To ensure Pk values are treated as unique identifiers (like strings), we'll convert them to string type
#low_prob_pks['Pk'] = low_prob_pks['Pk'].astype(str)

# Display the first few rows of low probability Pks for a glimpse
print(low_prob_pks.shape)
low_prob_pks.head()

In [ ]:
from scipy.spatial.distance import cdist

# Convert the vector lists into a DataFrame for distance calculations
vector_df = pd.DataFrame(low_prob_pks['Vector'].tolist(), index=low_prob_pks['Pk'])

# Compute the Manhattan (cityblock) distances between all pairs of low-probability Pks
manhattan_distances = cdist(vector_df, vector_df, metric='cityblock')

# Convert the distance matrix to a DataFrame for easier manipulation
distance_df = pd.DataFrame(manhattan_distances, index=vector_df.index, columns=vector_df.index)

# Display the first few rows of the distance matrix for a glimpse
print(distance_df.shape)
distance_df.head()

In [ ]:
# save the distance matrix to a csv file
distance_df.to_csv('distance_matrix.csv')

In [ ]:
# see if the distance_df and filtered_pk_correlation_matrix have the same column names
print(distance_df.columns.equals(filtered_pk_correlation_matrix.columns))

In [ ]:
# Ensuring that Pk values are consistent and exist in all dataframes

# Check if all Pks in combined_df are in the distance and correlation matrices
pks_in_combined_df = set(combined_df['Pk'])
pks_in_distance_matrix = set(distance_df.index)
pks_in_correlation_matrix = set(filtered_pk_correlation_matrix.index)

# Find Pks that are not common in all dataframes
non_common_pks = (pks_in_combined_df - pks_in_distance_matrix) | (pks_in_combined_df - pks_in_correlation_matrix)

# Display any non-common Pks
non_common_pks # the only excluded blocks


In [ ]:
# Filter the combined_df to include only the Pk values that are present in both the distance and correlation matrices
common_pks = pks_in_distance_matrix.intersection(pks_in_correlation_matrix)
print(len(common_pks))
filtered_combined_df = combined_df[combined_df['Pk'].isin(common_pks)]



In [ ]:
# Implementing the merging process

# Function to find the closest Pk for merging
def find_closest_pk(pk, distance_df, correlation_df):
    # Get distances and correlations for the given Pk
    distances = distance_df.loc[pk]
    correlations = correlation_df.loc[pk]

    # Ignore the distance to itself by setting it to infinity
    distances[pk] = float('inf')

    # Find the minimum distance
    min_distance = distances.min()
    closest_pks = distances[distances == min_distance].index

    # If more than one Pk is at the same minimum distance, choose the one with the highest correlation
    if len(closest_pks) > 1:
        closest_pk = correlations[closest_pks].idxmax()
    else:
        closest_pk = closest_pks[0]

    return closest_pk

# Merging function  for each cluster

def merge_clusters(data, distance_df, correlation_df, threshold_prob):
    merges = []
    pks_to_merge = set(data[data['Prob'] < threshold_prob]['Pk'])

    while pks_to_merge:
        # Select a random Pk to start merging
        current_pk = pks_to_merge.pop()
        merge_count = 0
        last_pk = None

        # Continue merging until the probability threshold is met or no more Pks to merge
        while data.loc[data['Pk'] == current_pk, 'Prob'].iloc[0] < threshold_prob and pks_to_merge:
            closest_pk = find_closest_pk(current_pk, distance_df, correlation_df)

            # Merge current Pk with the closest Pk
            current_prob = data.loc[data['Pk'] == current_pk, 'Prob'].iloc[0]
            closest_pk_prob = data.loc[data['Pk'] == closest_pk, 'Prob'].iloc[0]
            new_prob = current_prob + closest_pk_prob

            # Update merge count and last merged Pk
            merge_count += 1
            last_pk = closest_pk

            # Update the dataframe with new probability
            data.loc[data['Pk'] == current_pk, 'Prob'] = new_prob
            data.loc[data['Pk'] == closest_pk, 'Prob'] = new_prob

            # Remove the merged Pk from the set
            pks_to_merge.discard(closest_pk)

            # Update the current Pk if the closest Pk has a higher initial probability
            if closest_pk_prob > current_prob:
                current_pk = closest_pk

        # Record the merge details
        merges.append({
            'Birth Prob': current_prob,
            'State Index': current_pk,
            'Final Index': last_pk,
            'Pk': current_pk,
            'Prob': new_prob,
            'Merge Count': merge_count,
            'Last Pk': last_pk
        })

    return pd.DataFrame(merges)

In [ ]:
# threshold for the merge
threshold = 5/2638
# Perform the merging process
merged_data_full = merge_clusters(combined_df, distance_df, filtered_pk_correlation_matrix, threshold)

# Display the first few rows of the merge records
print(merged_data_full.shape)
merged_data_full.head(15)

In [ ]:
#find the maximum and lowest probability in merged data
max_prob = merged_data_full['Prob'].max()
min_prob = merged_data_full['Prob'].min()

max_prob, min_prob

In [ ]:
# value counts of the merge count
merged_data_full['Merge Count'].value_counts()

In [ ]:
# find the unique Pk in the merged data
unique_pk_merged = merged_data_full["Pk"].unique()
print(len(unique_pk_merged))